# Step 09: Function Calling with Auto-Invoke

This notebook demonstrates automatic function calling with plugins using FunctionChoiceBehavior.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

In [ ]:
import asyncio
import sys
from pathlib import Path
import logging

sys.path.insert(0, str(Path.cwd().parent))

In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.utils.logging import setup_logging
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    AzureChatPromptExecutionSettings,
)
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.connectors.ai import FunctionChoiceBehavior
from plugins.datetime import DateTimePlugin
from plugins.location import LocationPlugin
from plugins.weather import WeatherPlugin

In [ ]:
# Enable logging (optional)
setup_logging()
logging.getLogger("semantic_kernel").setLevel(logging.DEBUG)
logging.getLogger("kernel").setLevel(logging.DEBUG)

In [ ]:
# Initialize kernel and plugins
kernel = Kernel()
azure_chat_completion = AzureChatCompletion(instruction_role="system")
kernel.add_service(azure_chat_completion)

kernel.add_plugin(LocationPlugin(), plugin_name="location_plugin")
kernel.add_plugin(DateTimePlugin(), plugin_name="time_plugin")
kernel.add_plugin(WeatherPlugin(), plugin_name="weather_plugin")

In [ ]:
# Configure automatic function calling
execution_settings = AzureChatPromptExecutionSettings(
    function_choice_behavior=FunctionChoiceBehavior.Auto(
        auto_invoke=True,
        filters={
            "included_plugins": [
                "time_plugin",
                "location_plugin",
                "weather_plugin",
            ]
        },
    )
)

history = ChatHistory()

In [ ]:
# Interactive chat function
async def chat(user_input: str):
    history.add_user_message(user_input)
    result = await azure_chat_completion.get_chat_message_content(
        chat_history=history,
        settings=execution_settings,
        kernel=kernel,
    )
    print("Assistant > " + str(result))
    history.add_message(result)
    return result

In [ ]:
# Example: Ask about weather
await chat("What's the weather like?")

In [ ]:
# Interactive chat loop
# Type 'exit' to quit
while True:
    user_input = input("User > ")
    
    if user_input == "exit":
        break
    
    await chat(user_input)